In [1]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "bruteforce",
    "quality": "medium",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}
config = dim.Config(params)
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]

/home/francesco/dev/deep-image-matching/src/deep_image_matching/thirdparty/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
xFormers not available
xFormers not available


Deep Image Matching loaded in 5.334 seconds.


Run feature extraction and matching


In [2]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

Loaded SuperPoint model
2025-07-04 19:52:04 | [INFO    ] Running image matching with the following configuration:
2025-07-04 19:52:04 | [INFO    ]   Image folder: assets/example_cyprus/images
2025-07-04 19:52:04 | [INFO    ]   Output folder: assets/example_cyprus/results_superpoint+lightglue_bruteforce_quality_medium
2025-07-04 19:52:04 | [INFO    ]   Number of images: 10
2025-07-04 19:52:04 | [INFO    ]   Matching strategy: bruteforce
2025-07-04 19:52:04 | [INFO    ]   Image quality: MEDIUM
2025-07-04 19:52:04 | [INFO    ]   Tile selection: NONE
2025-07-04 19:52:04 | [INFO    ]   Feature extraction method: superpoint
2025-07-04 19:52:04 | [INFO    ]   Matching method: lightglue
2025-07-04 19:52:04 | [INFO    ]   Geometric verification: PYDEGENSAC
2025-07-04 19:52:04 | [INFO    ]   CUDA available: False
2025-07-04 19:52:04 | [INFO    ] Number of pairs: 45
2025-07-04 19:52:04 | [INFO    ] Found 45 pairs.
2025-07-04 19:52:04 | [INFO    ] Extracting features with superpoint...
2025-07-04 

100%|██████████| 10/10 [00:09<00:00,  1.04it/s]

2025-07-04 19:52:14 | [INFO    ] Features extracted!
2025-07-04 19:52:14 | [INFO    ] Matching features with lightglue...
2025-07-04 19:52:14 | [INFO    ] lightglue configuration: 
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}
2025-07-04 19:52:14 | [INFO    ] Matching features...
2025-07-04 19:52:14 | [INFO    ] 



100%|██████████| 45/45 [01:24<00:00,  1.88s/it]

2025-07-04 19:53:39 | [INFO    ] [Timer] | [matching] generate_pairs=1.647, extract_features=9.657, Match pair=1.713, Total execution=95.815
2025-07-04 19:53:39 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format


In [3]:
# First remove existing database if it exists
database_path = output_dir / "database.db"
if database_path.exists():
    database_path.unlink()

# Export to COLMAP format
dim.io.export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_config_path=config.general["camera_options"],
)

100%|██████████| 10/10 [00:00<00:00, 1049.78it/s]
45it [00:00, 1823.28it/s]             
 86%|████████▌ | 31/36 [00:00<00:00, 2622.81it/s]


Run reconstruction with pycolmap


In [4]:
# Optional - You can specify some reconstruction configuration
# reconst_opts = (
#     {
#         "ba_refine_focal_length": True,
#         "ba_refine_principal_point": False,
#         "ba_refine_extra_params": False,
#     },
# )
reconst_opts = {}
model = dim.reconstruction.incremental_reconstruction(
    database_path=output_dir / "database.db",
    image_dir=imgs_dir,
    sfm_dir=output_dir / "reconstruction",
    reconstruction_options=reconst_opts,
    refine_intrinsics=True,
    ignore_two_view_tracks=True,
    filter_min_tri_angle=None,
    export_ply=True,
    export_text=True,
    export_bundler=False,
)


2025-07-04 19:53:39 | [INFO    ] Running 3D reconstruction...

Images registered:   0%|                               |  0/10 [00:01<?, 0.00/s]

I20250704 19:53:41.283129 140427792920640 incremental_pipeline.cc:253] Loading database
I20250704 19:53:41.287739 140427792920640 database_cache.cc:66] Loading rigs...
I20250704 19:53:41.287803 140427792920640 database_cache.cc:76]  0 in 0.000s
I20250704 19:53:41.287844 140427792920640 database_cache.cc:84] Loading cameras...
I20250704 19:53:41.287925 140427792920640 database_cache.cc:102]  2 in 0.000s
I20250704 19:53:41.287944 140427792920640 database_cache.cc:110] Loading frames...
I20250704 19:53:41.287977 140427792920640 database_cache.cc:127]  0 in 0.000s
I20250704 19:53:41.287992 140427792920640 database_cache.cc:135] Loading matches...
I20250704 19:53:41.288703 140427792920640 database_cache.cc:140]  31 in 0.001s
I20250704 19:53:41.288727 140427792920640 database_cache.cc:156] Loading images...
I20250704 19:53:41.290382 140427792920640 database_cache.cc:241]  10 in 0.002s (connected 10)
I20250704 19:53:41.290413 140427792920640 database_cache.cc:252] Building correspondence grap

Images registered:  20%|█████▍                     |  2/10 [00:03<00:10, 0.76/s]

I20250704 19:53:42.902863 140427792920640 incremental_pipeline.cc:427] Registering image #7 (num_reg_frames=2)
I20250704 19:53:42.902922 140427792920640 incremental_pipeline.cc:430] => Image sees 640 / 1895 points
I20250704 19:53:43.168738 140427792920640 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment


Images registered:  30%|████████▏                  |  3/10 [00:04<00:10, 0.73/s]

...

Images registered:  70%|██████████████████▉        |  7/10 [00:11<00:05, 0.62/s]

I20250704 19:53:51.509364 140427792920640 incremental_pipeline.cc:427] Registering image #1 (num_reg_frames=7)
I20250704 19:53:51.509417 140427792920640 incremental_pipeline.cc:430] => Image sees 1056 / 1497 points
I20250704 19:53:51.984060 140427792920640 incremental_pipeline.cc:42] Retriangulation and Global bundle adjustment


Images registered:  80%|█████████████████████▋     |  8/10 [00:13<00:03, 0.60/s]

I20250704 19:53:53.590156 140427792920640 incremental_pipeline.cc:581] Keeping successful reconstruction
I20250704 19:53:53.592023 140427792920640 incremental_pipeline.cc:299] Finding good initial image pair
I20250704 19:53:53.592390 140427792920640 incremental_pipeline.cc:303] => No good initial image pair found.
I20250704 19:53:53.592425 140427792920640 incremental_pipeline.cc:544] Discarding reconstruction due to no initial pair
I20250704 19:53:53.592503 140427792920640 incremental_pipeline.cc:234] => Relaxing the initialization constraints.
I20250704 19:53:53.592926 140427792920640 incremental_pipeline.cc:299] Finding good initial image pair
I20250704 19:53:53.593250 140427792920640 incremental_pipeline.cc:303] => No good initial image pair found.
I20250704 19:53:53.593351 140427792920640 incremental_pipeline.cc:544] Discarding reconstruction due to no initial pair
I20250704 19:53:53.593405 140427792920640 incremental_pipeline.cc:243] => Relaxing the initialization constraints.
I20

2025-07-04 19:53:54 | [INFO    ] Reconstructed 1 model(s).
Images registered:  80%|█████████████████████▋     |  8/10 [00:13<00:03, 0.60/s]
2025-07-04 19:53:54 | [INFO    ] Largest model is #0 with 8 images.
